In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

'''
@author :wangkechn
https://github.com/wangkechn
'''

In [ ]:
#载入数据集
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

In [3]:
#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size


#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)#生成一个截断的正态分布
    return tf.Variable(initial)

#初始化偏置值
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层
def conv2d(x,W):
    #x input tensor of shape '[batch, in_height, in_width, in_channels]'
    #W filter / kernel tensor of shape [filter_height, filter_width, in_channels ,out_channels]
    #'strides[0] = strides[3]=1'.strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding：'A' string 'from : SAME VALID'
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')#2维的卷积操作


def max_pool_2x2(x):
    #ksize [1,x,y,1] 窗口大小
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#定义两个placeholder
x = tf.placeholder(tf.float32,[None,784])#28*28
y = tf.placeholder(tf.float32,[None,10])

#改变x的格式转为4D向量'[batch, in_height, in_width, in_channels]'
x_image = tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([5,5,1,32])#5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_variable([32])#每一个卷积核一个偏置值

#把x_image和权值向量进行卷积，再加上偏置，然后应用relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)#进行max-pooling

#初始化第二个卷积层的权值和偏置
W_conv2 = weight_variable([5,5,32,64])#5*5的采样窗口，64个卷积核从32个平面抽取特征
b_conv2 = bias_variable([64])#每一个卷积核一个偏置值

#把h_pool1和权值向量进行卷积，再加上偏置，然后应用relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)#进行max-pooling

#28*28的图片第一次卷积后还是28*28，第一次池化后变为14*14
#第二次卷积后为14*14,第二次池化后变为了7*7
#经过上面的操作后得到64张7*7的平面

#初始化第一个全连接层的权值
W_fc1 = weight_variable([7*7*64,1024])#上一层有7*7*64个神经元，全连接层有1024个神经元
b_fc1 = bias_variable([1024])#1024个节点

#把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

#keep_prob用来表示神经元的输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个全连接层
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)




#交叉熵代价函数  该版本的之后会被移除 详见下面的WARNING
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用adamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#初始化变量
init = tf.global_variables_initializer()


#equal 比较相同 返回True  结果存放在布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))  #argmax()返回一维向量中最大的值的索引位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #cast()转化格式 True为1.0 False:0.0 


with tf.Session() as sess:
    sess.run(init)
    #重新运行程序 否则图会重叠
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
                         
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print('Iter' + str(epoch) + ",Testing Accuracy "+ str(acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter0,Testing Accuracy 0.9527
Iter1,Testing Accuracy 0.9696
Iter2,Testing Accuracy 0.9762
Iter3,Testing Accuracy 0.9821
Iter4,Testing Accuracy 0.981
Iter5,Testing Accuracy 0.9852
Iter6,Testing Accuracy 0.9861
Iter7,Testing Accuracy 0.987
Iter8,Testing Accuracy 0.9874
Iter9,Testing Accuracy 0.989
Iter10,Testing Accuracy 0.9886
Iter11,Testing Accuracy 0.9899
Iter12,Testing Accuracy 0.9896
Iter13,Testing Accuracy 0.9897
Iter14,Testing Accuracy 0.9903
Iter15,Testing Accuracy 0.9892
Iter16,Testing Accuracy 0.9905
Iter17,Testing Accuracy 0.9915
Iter18,Testing Accuracy 0.9919
Iter19,Testing Accuracy 0.9913
Iter20,Testing Accuracy 0.9913
